In [1]:
import json
from StringIO import StringIO
import pandas as pd
import openpyxl
import pandas as pd
import os
import numpy as np
import datetime
import csv

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [4]:
listings = pd.read_csv('inside_airbnb/listings.csv')

In [5]:
special_events = pd.read_csv('Datasources/sdcounty_datasets/special_events_list_datasd.csv')

In [6]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [7]:
schema_two = StructType([StructField("event_id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [8]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [9]:
spql_special_events_info = sqlContext.createDataFrame(special_events[['event_id', 'longitude', 'latitude']].dropna(), schema_two)

In [10]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [11]:
spql_special_events_info_rdd = spql_special_events_info.rdd.map(lambda row:(row.event_id, row.longitude,row.latitude))

In [12]:
spql_listings_and_events_info_rdd = spql_listings_info_rdd.cartesian(spql_special_events_info_rdd)

In [13]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [14]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [15]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_rdd.map(distance_calculation)

In [16]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField("event_id", IntegerType(), True),
                    StructField("event_longitude", DoubleType(), True),
                    StructField("event_latitude", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [17]:
spql_listings_and_events_info_df = sqlContext.createDataFrame(spql_listings_and_events_info_rdd, schema_three)

In [18]:
spql_listings_and_events_info_rdd = spql_listings_and_events_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.event_id, row.event_longitude, row.event_latitude, row.distance))

In [19]:
closest_event_distance = spql_listings_and_events_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [20]:
closest_event_distance = spql_listings_and_events_info_rdd.cartesian(closest_event_distance).filter(lambda x: x[0][-1] == x[1][-1]).map(lambda x: (x[0][0], x[0][-1], x[0][3]))

In [21]:
close_events_count_one = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 1).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_three = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 3).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_five = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 5).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_events_count_ten = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 10).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [22]:
close_events_count_sixteen = spql_listings_and_events_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [23]:
count = 0

In [24]:
import datetime
today = datetime.date.today()
count+=1
filename = 'Datasources/listings_events/listings_events_' + str(today) + '_V' + str(count) + '.csv'

In [25]:
print filename

Datasources/listings_events/listings_events_2018-05-21_V1.csv


In [26]:
df1 = close_events_count_one.toDF()

In [27]:
df2 = close_events_count_three.toDF()

In [28]:
df3 = close_events_count_five.toDF()

In [29]:
df4 = close_events_count_ten.toDF()

In [30]:
df5 = close_events_count_sixteen.toDF()

In [31]:
df6 = closest_event_distance.toDF()

In [32]:
df1 = df1.toPandas()

In [33]:
df1 = df1.rename(columns={'_1':'listing_id', '_2':'event_count_1km'})

In [34]:
df2 = df2.toPandas()

In [35]:
df2 = df2.rename(columns={'_1':'listing_id', '_2':'event_count_3km'})

In [36]:
df3 = df3.toPandas()

In [37]:
df3 = df3.rename(columns={'_1':'listing_id', '_2':'event_count_5km'})

In [38]:
df4 = df4.toPandas()

In [39]:
df4 = df4.rename(columns={'_1':'listing_id', '_2':'event_count_10km'})

In [40]:
df5 = df5.toPandas()

In [41]:
df5 = df5.rename(columns={'_1':'listing_id', '_2':'event_count_16km'})

In [42]:
df6 = df6.toPandas()

Py4JJavaError: An error occurred while calling o303.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 23.0 failed 1 times, most recent failure: Lost task 2.0 in stage 23.0 (TID 107, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 208325 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1599)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1587)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1586)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1586)
	at org.apache.spar

In [ ]:
df6 = df6.rename(columns={'_1':'listing_id', '_2':'closest_distance_of_events', '_3': 'event_id_of_closest_distance'})

In [ ]:
t1 = df1.merge(df2, how='inner', on='listing_id')

In [ ]:
t2 = t1.merge(df3, how='inner', on='listing_id')
t3 = t2.merge(df4, how='inner', on='listing_id')

In [ ]:
t4 = t3.merge(df5, how='inner', on='listing_id')
t5 = t4.merge(df6, how='inner', on='listing_id')

In [ ]:
t5.head()

In [ ]:
t5 = t5.merge(special_events[['event_title', 'event_id', 'event_type']], how='outer', left_on='event_id_of_closest_distance', right_on = 'event_id')

In [ ]:
t5 = t5.dropna()

In [ ]:
t5.to_csv(filename,index=False)

In [ ]:
t5.head()

In [ ]:
sc.stop()